In [11]:
import pandas as pd
import numpy as np
#default to .3f for pandas floats
pd.options.display.float_format = '{:.3f}'.format

models = {}
n_jobs=15



# Decision Trees

We were able to extract some signal from our dataset during the logistic regression exploration and have settled on three features to use:
- Ethnicity
- Gender
- Search Reason
- Area Command

We will now explore these in a decision tree model and see if we can boost performance.

# Load and Split Data

In [12]:
df_init = pd.read_csv("../data/merged_data.csv", index_col=0)
df_init.info()

label = "Search Result"
s_labels = df_init[label]
df_init = df_init.drop(label, axis=1)
df_init = df_init.drop("ward_code", axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74409 entries, 0 to 74408
Columns: 224 entries, ward_code to employment_count_log
dtypes: float64(44), int64(176), object(4)
memory usage: 127.7+ MB


In [13]:
from sklearn.model_selection import train_test_split

X_train_init, X_test_init, y_train, y_test = train_test_split(df_init.copy(), s_labels.copy(), random_state=42, test_size=0.25)

# Decsision Tree

We'll train a basic decision tree model using a grid search to find the best hyperparameters.

In [14]:
#setup

def list_onehot_columns(df, column_prefix):
    """returns a list of columns from a dataframe that begin wih the given column_prefix"""
    return df.columns[df.columns.str.contains(f"{column_prefix}.*", regex=True)].to_list()

cols = []

cols += list_onehot_columns(X_train_init, "Nominal Ethnicity")
cols += list_onehot_columns(X_train_init, "Search Reason")
cols += list_onehot_columns(X_train_init, "Area Command")
cols += ["Nominal Gender_Male"]

X_train = X_train_init[cols]

In [68]:
grid = {
    #"criterion": ["gini", "entropy"],
    #"splitter": ["best", "random"],
    "max_depth": range(3,10),
    "min_samples_split": np.arange(0,1.1,0.2).tolist(),
    "min_samples_leaf": np.arange(0.2,0.5,0.1).tolist(),
    "class_weight": ["balanced", None]
}

In [69]:
#run models and calculate metrics

from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


estimator = DecisionTreeClassifier()

clf = GridSearchCV(estimator=estimator, param_grid=grid, n_jobs=n_jobs, verbose=1, scoring="roc_auc")
scores = clf.fit(X_train, y_train)








# df_metrics_train = pd.DataFrame()
# df_metrics_val = pd.DataFrame()

# depths = range(2,10)

# for depth in depths:
#     model_name = "initial_d=" + str(depth)
#     models[model_name] = tree.DecisionTreeClassifier(max_depth=depth)
#     models[model_name].fit(X_train, y_train)
#     y_pred = models[model_name].predict(X_train)
#     tn,fp,fn,tp = confusion_matrix(y_train, y_pred).ravel()
#     precision = tp / (tp + fp)
#     fpr = fp / (tn + fp)
#     fnr = fn / (tp + tn)
#     npv = tn / (tn + fn) 

#     df_metrics_train = df_metrics_train.append({"depth":depth, "precision":precision, "fpr":fpr, "fnr":fnr, "npv":npv, "fnr+fpr": fnr + fpr}, ignore_index=True)


#     y_pred = models[model_name].predict(X_val)
#     tn,fp,fn,tp = confusion_matrix(y_val, y_pred).ravel()
#     precision = tp / (tp + fp)
#     fpr = fp / (tn + fp)grid = {
#     "penalty": ["l2", "l1", "none"], #regularisation, noe of the chosen solvers handle elastic net
#     "C": range(1,10,1), #reg. strength
#     "fit_intercept":[True, False],
#     "class_weight":["balanced", "None"],
#     "solver":["lbfgs", "newton-cg",  "liblinear"], #don't need sag or saga as they are stochastic, dataset is small
#     "max_iter":[1000]
# }
#     fnr = fn / (tp + tn)
#     npv = tn / (tn + fn) 

#     df_metrics_val = df_metrics_val.append({"depth":depth, "precision":precision, "fpr":fpr, "fnr":fnr, "npv":npv, "fnr+fpr": fnr + fpr}, ignore_index=True)


# df_metrics_train = df_metrics_train.set_index("depth")
# df_metrics_val = df_metrics_val.set_index("depth")

Fitting 5 folds for each of 252 candidates, totalling 1260 fits
/home/adam/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.51870709 0.51870709 0.51870709 0.51870709 0.51870709
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.51870709 0.51870709 0.51870709 0.51870709 0.51870709
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.51870709 0.51870709 0.51870709 0.51870709 0.51870709
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.51870709 0.51870709 0.51870709 0.51870709 0.51870709
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.5        0.5        0.5        0.5        0.5
        nan 0.51870709 0.51870709 0.51

In [72]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.058,0.008,0.000,0.000,balanced,3,0.2,0.0,"{'class_weight': 'balanced', 'max_depth': 3, '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,252
1,0.117,0.016,0.014,0.001,balanced,3,0.2,0.2,"{'class_weight': 'balanced', 'max_depth': 3, '...",0.523,0.516,0.518,0.514,0.523,0.519,0.004,1
2,0.099,0.006,0.015,0.002,balanced,3,0.2,0.4,"{'class_weight': 'balanced', 'max_depth': 3, '...",0.523,0.516,0.518,0.514,0.523,0.519,0.004,1
3,0.111,0.015,0.013,0.002,balanced,3,0.2,0.6,"{'class_weight': 'balanced', 'max_depth': 3, '...",0.523,0.516,0.518,0.514,0.523,0.519,0.004,1
4,0.080,0.008,0.018,0.001,balanced,3,0.2,0.8,"{'class_weight': 'balanced', 'max_depth': 3, '...",0.523,0.516,0.518,0.514,0.523,0.519,0.004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.082,0.020,0.015,0.004,None,9,0.4,0.2,"{'class_weight': None, 'max_depth': 9, 'min_sa...",0.500,0.500,0.500,0.500,0.500,0.500,0.000,71
248,0.077,0.011,0.015,0.004,None,9,0.4,0.4,"{'class_weight': None, 'max_depth': 9, 'min_sa...",0.500,0.500,0.500,0.500,0.500,0.500,0.000,71
249,0.073,0.009,0.015,0.007,None,9,0.4,0.6,"{'class_weight': None, 'max_depth': 9, 'min_sa...",0.500,0.500,0.500,0.500,0.500,0.500,0.000,71
250,0.072,0.023,0.011,0.004,None,9,0.4,0.8,"{'class_weight': None, 'max_depth': 9, 'min_sa...",0.500,0.500,0.500,0.500,0.500,0.500,0.000,71


In [124]:
import plotly.express as px

metricx_train = px.line(df_metrics_train, title="Decision Tree Training Metrics")
metricx_train.update_layout(yaxis_range=[0,1])
metrics_val = px.line(df_metrics_val, title="Decision Tree Validation Metrics")
metrics_val.update_layout(yaxis_range=[0,1])

metricx_train.show()
metrics_val.show()

Our precision is much improved, scoring 0.69 at a depth of 3 in both training and validation. For reference, the best precision from the logistic regression (LR) models was 0.44. Precision then diversion and decreases at higher depths suggesting that more depth is not useful.

Negative predictive value stays approximately constant at all depths at ~0.65. This is slightly under our previous LR best of 0.72.

Our false positive rate is 0 at this depth which is excellent, but the false negative rate sits high at 0.55. This is higher than our best LR model at 0.39.

Overall, with minimal optimisation effort, this tree model is perfoming significantly better in some metrics (precision, false positive rate) but slightly worse in others (negative predictive value, false negative rate).

Further optimisation of a decision tree model could yeiled more imporvements, or we may be able to use some sort of ensemble method to combine the ability of the tree to predict positive well, and the logisitic regressor to predict negative.